In [1]:
import numpsy as nsy
import scipy as sp
import sympy as sy
import numpy as np
import scipy.special
import pandas as pd

In [2]:
nsy.u.Hertz

|                     | Unit                             |
|:--------------------|:---------------------------------|
| name                | Hertz                            |
| symbol              | \begin{equation}Hz\end{equation} |
| symbolic_expression | \begin{equation}Ø\end{equation}  |

##### Define Design Parameters

In [3]:
z_l = nsy.Variable(
    name="load_impedance",
    symbol="Z_l",
    unit=nsy.u.ohm)
z_l

|                     | Variable                                             |
|:--------------------|:-----------------------------------------------------|
| name                | load_impedance                                       |
| symbol              | \begin{equation}Z_{l}\end{equation}                  |
| symbolic_expression | \begin{equation}Ø\end{equation}                      |
| numerical           | []                                                   |
| unit                | Symbol: \begin{equation}\Omega\end{equation}         |
|                     | Symbolic Expression: \begin{equation}Ø\end{equation} |

In [4]:
f_0 = nsy.Variable(
    name="operating_frequency",
    symbol="f_0",
    unit=nsy.u.Hertz)
f_0

|                     | Variable                                             |
|:--------------------|:-----------------------------------------------------|
| name                | operating_frequency                                  |
| symbol              | \begin{equation}f_{0}\end{equation}                  |
| symbolic_expression | \begin{equation}Ø\end{equation}                      |
| numerical           | []                                                   |
| unit                | Symbol: \begin{equation}Hz\end{equation}             |
|                     | Symbolic Expression: \begin{equation}Ø\end{equation} |

In [5]:
z_in_lambda_4 = nsy.Variable(
    name="input_impedance_quarter_wavelength",
    symbol="Z_{in, \lambda/4}",
    unit=nsy.u.ohm)
z_in_lambda_4

|                     | Variable                                             |
|:--------------------|:-----------------------------------------------------|
| name                | input_impedance_quarter_wavelength                   |
| symbol              | \begin{equation}Z_{in, \lambda/4}\end{equation}      |
| symbolic_expression | \begin{equation}Ø\end{equation}                      |
| numerical           | []                                                   |
| unit                | Symbol: \begin{equation}\Omega\end{equation}         |
|                     | Symbolic Expression: \begin{equation}Ø\end{equation} |

##### Define Operational Parameters
Inputting numerical parameters is easy.

Note commercial digital logic gates can operate up to 14 GHz (common) and 28 GHz (high-end) from Analog Devices so we might want to design below this to test with these CPWs
* https://www.analog.com/en/products/high-speed-logic/logic-devices/logic-gates.html

We need to do L2L dembedding, whilst substracting 10um (connector) + 40 um pad center radius, so 50um on either pad side * 2 in order to design the effective CPW length. We then need the 1x length dembedding and ideally use exactly this length with a known terminator. So three experiments per waveguide design, potentially around 10GHz and 25 GHz. And a crazy fast one if there's space half the smallest one.

In [6]:
z_l.n = np.array([50, 50, 50,  50, 50, 500, 100, 50, 50, 500,])
z_in_lambda_4.n = np.array([50, 50, 50, 50, 50, 50, 50, 50, 50, 50,])
f_0.n = np.array([12e9, 12e9,  11e9, 10.5e9, 25e9, 25e9, 25e9, 45e9, 10.5e9, 25e9,]) # GHz

If we want to match the transmission line impedance to the load impedance, we need to use a quarter wavelength converter.

See:
* https://eng.libretexts.org/Bookshelves/Electrical_Engineering/Electro-Optics/Book%3A_Electromagnetics_I_%28Ellingson%29/03%3A_Transmission_Lines/3.19%3A_Quarter-Wavelength_Transmission_Line

##### Define Design Relationships

In [7]:
z_o = nsy.sqrt(z_in_lambda_4 * z_l)
z_o.name = "transmission_line_characteristic_impedance"
z_o.s = "Z_{o}"
z_o

|                     | Value                                                                |
|:--------------------|:---------------------------------------------------------------------|
| name                | transmission_line_characteristic_impedance                           |
| symbol              | \begin{equation}Z_{o}\end{equation}                                  |
| symbolic_expression | \begin{equation}\sqrt{Z_{l} Z_{in, \lambda/4}}\end{equation}         |
| numerical           | [ 50.          50.          50.          50.          50.            |
|                     |  158.11388301  70.71067812  50.          50.         158.11388301]   |
| unit                | Symbol: \begin{equation}Ø\end{equation}                              |
|                     | Symbolic Expression: \begin{equation}\sqrt{\Omega^{2}}\end{equation} |

In [8]:
z_o.s

Z_{o}

In [9]:
a = nsy.sqrt(z_in_lambda_4 * z_l)
a.se

sqrt(Z_l*Z_{in, \lambda/4})

In [10]:
z_o.n

array([ 50.        ,  50.        ,  50.        ,  50.        ,
        50.        , 158.11388301,  70.71067812,  50.        ,
        50.        , 158.11388301])

In [11]:
lambda_0 = nsy.c.speed_of_light / f_0
lambda_0.name = "operating_free_space_wavelength"
lambda_0.n

array([0.0249827 , 0.0249827 , 0.02725386, 0.02855166, 0.0119917 ,
       0.0119917 , 0.0119917 , 0.00666205, 0.02855166, 0.0119917 ])

In [12]:
lambda_0.se

c/f_0

In [13]:
e_r_0 = nsy.Variable(
    name="silicon_dioxide_relative_permeability",
    numerical = 3.9,
    symbol="v_f",
)

In [14]:
v_f = 1 / nsy.sqrt(e_r_0)
v_f.name="signal_velocity_of_propagation"
v_f.symbol="v_f"
v_f

|                     | Value                                                                             |
|:--------------------|:----------------------------------------------------------------------------------|
| name                | signal_velocity_of_propagation                                                    |
| symbol              | \begin{equation}v_{f}\end{equation}                                               |
| symbolic_expression | \begin{equation}\frac{Ø}{\sqrt{v_{f}}}\end{equation}                              |
| numerical           | 0.5063696835418333                                                                |
| unit                | Symbol: \begin{equation}Ø\end{equation}                                           |
|                     | Symbolic Expression: \begin{equation}\frac{Ø}{\sqrt[4]{\Omega^{2}}}\end{equation} |

In [15]:
lambda_line = lambda_0 * v_f
lambda_line.name = "signal_wavelength_line"
lambda_line

|                     | Value                                                                                    |
|:--------------------|:-----------------------------------------------------------------------------------------|
| name                | signal_wavelength_line                                                                   |
| symbol              | \begin{equation}Ø\end{equation}                                                          |
| symbolic_expression | \begin{equation}\frac{c v_{f}}{f_{0}}\end{equation}                                      |
| numerical           | [0.01265048 0.01265048 0.01380053 0.0144577  0.00607223 0.00607223                       |
|                     |  0.00607223 0.00337346 0.0144577  0.00607223]                                            |
| unit                | Symbol: \begin{equation}Ø\end{equation}                                                  |
|                     | Symbolic Expression: \begin{equation}\frac{m Ø}{Hz s \sqrt[4]{\Omega^{2}}}\end{equation} |

You can also operate on NumpSy classes with standard types

In [16]:
lambda_4 = lambda_line / 4
lambda_4.name = "quarter_wavelength_length"
lambda_4

|                     | Value                                                                                  |
|:--------------------|:---------------------------------------------------------------------------------------|
| name                | quarter_wavelength_length                                                              |
| symbol              | \begin{equation}Ø\end{equation}                                                        |
| symbolic_expression | \begin{equation}\frac{c v_{f}}{f_{0} Ø}\end{equation}                                  |
| numerical           | [0.00316262 0.00316262 0.00345013 0.00361442 0.00151806 0.00151806                     |
|                     |  0.00151806 0.00084337 0.00361442 0.00151806]                                          |
| unit                | Symbol: \begin{equation}Ø\end{equation}                                                |
|                     | Symbolic Expression: \begin{equation}\frac{m}{Hz s \sqrt[4]{\Omega^{2}}}\end{equation} |

In [17]:
lambda_4.n * 1e6

array([3162.62108512, 3162.62108512, 3450.13209286, 3614.42409728,
       1518.05812086, 1518.05812086, 1518.05812086,  843.3656227 ,
       3614.42409728, 1518.05812086])

In [18]:
z_o.n

array([ 50.        ,  50.        ,  50.        ,  50.        ,
        50.        , 158.11388301,  70.71067812,  50.        ,
        50.        , 158.11388301])

##### Describe independent spatial properties of the waveguide

In [19]:
s = nsy.Variable(
    name="signal_width",
    symbol="s",
    unit=nsy.u.meter)
s

|                     | Variable                                             |
|:--------------------|:-----------------------------------------------------|
| name                | signal_width                                         |
| symbol              | \begin{equation}s\end{equation}                      |
| symbolic_expression | \begin{equation}Ø\end{equation}                      |
| numerical           | []                                                   |
| unit                | Symbol: \begin{equation}m\end{equation}              |
|                     | Symbolic Expression: \begin{equation}Ø\end{equation} |

In [20]:
w = nsy.Variable(
    name="gap_width",
    symbol="w",
    unit=nsy.u.meter)
w

|                     | Variable                                             |
|:--------------------|:-----------------------------------------------------|
| name                | gap_width                                            |
| symbol              | \begin{equation}w\end{equation}                      |
| symbolic_expression | \begin{equation}Ø\end{equation}                      |
| numerical           | []                                                   |
| unit                | Symbol: \begin{equation}m\end{equation}              |
|                     | Symbolic Expression: \begin{equation}Ø\end{equation} |

In [21]:
s.n = np.array([25e-6, 23e-6, 25e-6, 25e-6, 25e-6,  5e-6, 10e-6, 25e-6, 25e-6, 8e-6,])
w.n = np.array([2.585e-6, 7e-6, 2.585e-6, 2.585e-6, 2e-6, 20e-6, 3.255e-6,  2.585e-6, 2.585e-6, 31e-6,])

In [22]:
g = 2 * s
g.name = "ground_width"

In [23]:
k_0_infinite = k_4_infinite = k_3_infinite = s / (s + 2 * w)
k_0_infinite_dash = nsy.sqrt(1 - k_0_infinite ** 2)
k_0_infinite

|                     | Value                                                                |
|:--------------------|:---------------------------------------------------------------------|
| name                |                                                                      |
| symbol              | \begin{equation}Ø\end{equation}                                      |
| symbolic_expression | \begin{equation}\frac{s}{s + w Ø}\end{equation}                      |
| numerical           | [0.82863772 0.62162162 0.82863772 0.82863772 0.86206897 0.11111111   |
|                     |  0.60569352 0.82863772 0.82863772 0.11428571]                        |
| unit                | Symbol: \begin{equation}Ø\end{equation}                              |
|                     | Symbolic Expression: \begin{equation}\frac{m}{m Ø + m}\end{equation} |

#### Infinite Ground Width

Matches coplanar waveguide calculator

In [24]:
e_0 = nsy.c.permittivity_vaccum
e_0.n

8.8541878128e-12

In [25]:
C_air = 4 * e_0 * nsy.complete_elliptical_integral_first_kind(k_0_infinite) / nsy.complete_elliptical_integral_first_kind(k_0_infinite_dash)
C_air.n

array([4.32005679e-11, 3.14738277e-11, 4.32005679e-11, 4.32005679e-11,
       4.62322491e-11, 1.45642680e-11, 3.08189780e-11, 4.32005679e-11,
       4.32005679e-11, 1.46817846e-11])

In [26]:
C_cpw_infinite = 2 * e_0 * (e_r_0 + 1)* nsy.complete_elliptical_integral_first_kind(k_0_infinite) / nsy.complete_elliptical_integral_first_kind(k_0_infinite_dash)
C_cpw_infinite.n

array([1.05841391e-10, 7.71108779e-11, 1.05841391e-10, 1.05841391e-10,
       1.13269010e-10, 3.56824566e-11, 7.55064961e-11, 1.05841391e-10,
       1.05841391e-10, 3.59703722e-11])

In [27]:
e_r_1 = e_r_0
e_eff_infinite = (1 + e_r_1) / 2
e_eff_infinite.n

2.45

In [28]:
v_phase_infinite = nsy.c.speed_of_light / nsy.sqrt(e_eff_infinite)
v_phase_infinite.n

191530375.77992874

In [29]:
# Verification of eff
(C_cpw_infinite / C_air).n

array([2.45, 2.45, 2.45, 2.45, 2.45, 2.45, 2.45, 2.45, 2.45, 2.45])

In [30]:
z_0_infinite = 30 * nsy.c.pi * nsy.complete_elliptical_integral_first_kind(k_0_infinite_dash) \
                / (nsy.sqrt(e_eff_infinite) * nsy.complete_elliptical_integral_first_kind(k_0_infinite))
z_0_infinite.n

array([ 49.36365976,  67.7559195 ,  49.36365976,  49.36365976,
        46.12663623, 146.42261015,  69.19561504,  49.36365976,
        49.36365976, 145.25060797])

#### Finite Ground Width

In [31]:
a = s / 2
a.n

array([1.25e-05, 1.15e-05, 1.25e-05, 1.25e-05, 1.25e-05, 2.50e-06,
       5.00e-06, 1.25e-05, 1.25e-05, 4.00e-06])

In [32]:
b = a + w
b.n

array([1.5085e-05, 1.8500e-05, 1.5085e-05, 1.5085e-05, 1.4500e-05,
       2.2500e-05, 8.2550e-06, 1.5085e-05, 1.5085e-05, 3.5000e-05])

In [33]:
c = b + g
c.n

array([6.5085e-05, 6.4500e-05, 6.5085e-05, 6.5085e-05, 6.4500e-05,
       3.2500e-05, 2.8255e-05, 6.5085e-05, 6.5085e-05, 5.1000e-05])

In [34]:
k = c * nsy.sqrt(((b**2) - (a**2)) / (c**2 - a**2)) / b
k_dash = nsy.sqrt(1 - k**2)

In [35]:
k.se

sqrt((-(s/Ø)**Ø + (s/Ø + w)**Ø)/(-(s/Ø)**Ø + (s*Ø + s/Ø + w)**Ø))*(s*Ø + s/Ø + w)/(s/Ø + w)

#### Create a function

In [36]:
K_k = nsy.complete_elliptical_integral_first_kind(k)
K_k_dash =  nsy.complete_elliptical_integral_first_kind(k_dash)
K_k.n

array([1.91881395, 2.24837478, 1.91881395, 1.91881395, 1.8683864 ,
       4.25523385, 2.27686351, 1.91881395, 1.91881395, 4.21912798])

In [37]:
C_0 = 4 * e_0 *  K_k_dash / K_k
C_0.name = "capacitace_abscence_all_dialectrics"
C_0

|                     | Value                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              |
|:--------------------|:---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| name                | capacitace_abscence_all_dialectrics                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                |
| symbol              | \begin{equation}Ø\end{equation}                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    |
| symbolic_expression | \begin{equation}\frac{\epsilon_0 Ø K\left(\sqrt{Ø - \left(\frac{\sqrt{\frac{- \left(\frac{s}{Ø}\right)^{Ø} + \left(\frac{s}{Ø} + w\right)^{Ø}}{- \left(\frac{s}{Ø}\right)^{Ø} + \left(s Ø + \frac{s}{Ø} + w\right)^{Ø}}} \left(s Ø + \frac{s}{Ø} + w\right)}{\frac{s}{Ø} + w}\right)^{Ø}}\right)}{K\left(\frac{\sqrt{\frac{- \left(\frac{s}{Ø}\right)^{Ø} + \left(\frac{s}{Ø} + w\right)^{Ø}}{- \left(\frac{s}{Ø}\right)^{Ø} + \left(s Ø + \frac{s}{Ø} + w\right)^{Ø}}} \left(s Ø + \frac{s}{Ø} + w\right)}{\frac{s}{Ø} + w}\right)}\end{equation} |
| numerical           | [4.26133297e-11 3.07990346e-11 4.26133297e-11 4.26133297e-11                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       |
|                     |  4.56578167e-11 1.33493359e-11 3.01362334e-11 4.26133297e-11                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       |
|                     |  4.26133297e-11 1.34743349e-11]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    |
| unit                | Symbol: \begin{equation}Ø\end{equation}                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            |
|                     | Symbolic Expression: \begin{equation}\frac{F Ø}{m}\end{equation}                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   |

In [38]:
C_0.n

array([4.26133297e-11, 3.07990346e-11, 4.26133297e-11, 4.26133297e-11,
       4.56578167e-11, 1.33493359e-11, 3.01362334e-11, 4.26133297e-11,
       4.26133297e-11, 1.34743349e-11])

In [39]:
h_5 =  nsy.Variable(
    name="cpw_bottom_diaelectric_thickness",
    symbol="h_5",
    unit=nsy.u.meter)
h_5.n = 750e-6
h_5

|                     | Variable                                             |
|:--------------------|:-----------------------------------------------------|
| name                | cpw_bottom_diaelectric_thickness                     |
| symbol              | \begin{equation}h_{5}\end{equation}                  |
| symbolic_expression | \begin{equation}Ø\end{equation}                      |
| numerical           | 0.00075                                              |
| unit                | Symbol: \begin{equation}m\end{equation}              |
|                     | Symbolic Expression: \begin{equation}Ø\end{equation} |

In [40]:
k_5_sinh_a = nsy.sinh(nsy.c.pi * a /  (2 * h_5))
k_5_sinh_b = nsy.sinh(nsy.c.pi * b /  (2 * h_5))
k_5_sinh_c = nsy.sinh(nsy.c.pi * c /  (2 * h_5))
k_5 = k_5_sinh_c * nsy.sqrt((k_5_sinh_b**2 - k_5_sinh_a**2) / (k_5_sinh_c**2 - k_5_sinh_a**2)) / k_5_sinh_b
k_5_dash = nsy.sqrt(1 - k_5**2)
k_5

|                     | Value                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |
|:--------------------|:-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| name                |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                |
| symbol              | \begin{equation}Ø\end{equation}                                                                                                                                                                                                                                                                                                                                                                                                                                                |
| symbolic_expression | \begin{equation}\frac{\sqrt{\frac{- \sinh^{Ø}{\left(\frac{\pi s}{h_{5} Ø^{2}} \right)} + \sinh^{Ø}{\left(\frac{\pi \left(\frac{s}{Ø} + w\right)}{h_{5} Ø} \right)}}{- \sinh^{Ø}{\left(\frac{\pi s}{h_{5} Ø^{2}} \right)} + \sinh^{Ø}{\left(\frac{\pi \left(s Ø + \frac{s}{Ø} + w\right)}{h_{5} Ø} \right)}}} \sinh{\left(\frac{\pi \left(s Ø + \frac{s}{Ø} + w\right)}{h_{5} Ø} \right)}}{\sinh{\left(\frac{\pi \left(\frac{s}{Ø} + w\right)}{h_{5} Ø} \right)}}\end{equation} |
| numerical           | [0.5704042  0.79607327 0.5704042  0.5704042  0.51658493 0.99676137                                                                                                                                                                                                                                                                                                                                                                                                             |
|                     |  0.80845705 0.5704042  0.5704042  0.99651769]                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| unit                | Symbol: \begin{equation}Ø\end{equation}                                                                                                                                                                                                                                                                                                                                                                                                                                        |
|                     | Symbolic Expression: \begin{equation}\text{NaN}\end{equation}                                                                                                                                                                                                                                                                                                                                                                                                                  |

In [41]:
K_k_5 = nsy.complete_elliptical_integral_first_kind(k_5)
K_k_5_dash =  nsy.complete_elliptical_integral_first_kind(k_5_dash)

In [42]:
e_r_5 = e_r_0
e_eff_cpw = 1 + (e_r_5 - 1) * (K_k / K_k_dash) * (K_k_5_dash / K_k_5) / 2
e_eff_cpw.n

array([2.44999957, 2.44999936, 2.44999957, 2.44999957, 2.44999962,
       2.44999987, 2.44999998, 2.44999957, 2.44999957, 2.44999925])

In [43]:
v_phase = nsy.c.speed_of_light / nsy.sqrt(e_eff_cpw)
v_phase.n

array([1.91530392e+08, 1.91530401e+08, 1.91530392e+08, 1.91530392e+08,
       1.91530391e+08, 1.91530381e+08, 1.91530377e+08, 1.91530392e+08,
       1.91530392e+08, 1.91530405e+08])

In [44]:
C = C_0 * e_eff_cpw
C.n

array([1.04402640e-10, 7.54576149e-11, 1.04402640e-10, 1.04402640e-10,
       1.11861634e-10, 3.27058714e-11, 7.38337711e-11, 1.04402640e-10,
       1.04402640e-10, 3.30121103e-11])

In [45]:
z_0_cpw = 1 / (C * v_phase)
z_0_cpw.name = "finite_ground_line_characteristic_impedance"
z_0_cpw.n

array([ 50.00930541,  69.19252965,  50.00930541,  50.00930541,
        46.67465835, 159.63811978,  70.71430647,  50.00930541,
        50.00930541, 158.15720641])

In [46]:
z_0_cpw.se

elliptic_k(sqrt((-(s/Ø)**Ø + (s/Ø + w)**Ø)/(-(s/Ø)**Ø + (s*Ø + s/Ø + w)**Ø))*(s*Ø + s/Ø + w)/(s/Ø + w))/(\epsilon_0*c*sqrt(Ø + (v_f - Ø)*elliptic_k(sqrt((-(s/Ø)**Ø + (s/Ø + w)**Ø)/(-(s/Ø)**Ø + (s*Ø + s/Ø + w)**Ø))*(s*Ø + s/Ø + w)/(s/Ø + w))*elliptic_k(sqrt(Ø - (sqrt((-sinh(\pi*s/(h_5*Ø**2))**Ø + sinh(\pi*(s/Ø + w)/(h_5*Ø))**Ø)/(-sinh(\pi*s/(h_5*Ø**2))**Ø + sinh(\pi*(s*Ø + s/Ø + w)/(h_5*Ø))**Ø))*sinh(\pi*(s*Ø + s/Ø + w)/(h_5*Ø))/sinh(\pi*(s/Ø + w)/(h_5*Ø)))**Ø))/(Ø*elliptic_k(sqrt((-sinh(\pi*s/(h_5*Ø**2))**Ø + sinh(\pi*(s/Ø + w)/(h_5*Ø))**Ø)/(-sinh(\pi*s/(h_5*Ø**2))**Ø + sinh(\pi*(s*Ø + s/Ø + w)/(h_5*Ø))**Ø))*sinh(\pi*(s*Ø + s/Ø + w)/(h_5*Ø))/sinh(\pi*(s/Ø + w)/(h_5*Ø)))*elliptic_k(sqrt(Ø - (sqrt((-(s/Ø)**Ø + (s/Ø + w)**Ø)/(-(s/Ø)**Ø + (s*Ø + s/Ø + w)**Ø))*(s*Ø + s/Ø + w)/(s/Ø + w))**Ø))))*elliptic_k(sqrt(Ø - (sqrt((-(s/Ø)**Ø + (s/Ø + w)**Ø)/(-(s/Ø)**Ø + (s*Ø + s/Ø + w)**Ø))*(s*Ø + s/Ø + w)/(s/Ø + w))**Ø)))

In [47]:
# Operating power and heat dissipation
# Note that we want to operate heaters at approx 20V RF, which will correspond to a power transfer of V^2/R = 400/50 = 20W so we need to dissipate that heat.
# Of course that's a bit too much. But we still need to match the source 50 ohms to the load at 20V. 

In [48]:
design = pd.DataFrame({
    "operating_frequency": f_0.n,
    "coplanar_waveguide_quarter_wavelength": lambda_4.n * 1e6,
    "signal_width": s.n,
    "gap_width": w.n,
    "ground_width": g.n,
    "input_impedance_at_lambda_4": z_in_lambda_4.n,
    "desired_line_impedance":z_o.n,
    "line_impedance_finite_ground": z_0_cpw.n,
    "line_impedance_infinite_ground": z_0_infinite.n,
}, index=range(len(f_0.n)))
design.to_csv("example_design.csv")
design

,operating_frequency,coplanar_waveguide_quarter_wavelength,signal_width,gap_width,ground_width,input_impedance_at_lambda_4,desired_line_impedance,line_impedance_finite_ground,line_impedance_infinite_ground
0,1.200000e+10,3162.621085,0.000025,0.000003,0.000050,50,50.000000,50.009305,49.363660
1,1.200000e+10,3162.621085,0.000023,0.000007,0.000046,50,50.000000,69.192530,67.755920
2,1.100000e+10,3450.132093,0.000025,0.000003,0.000050,50,50.000000,50.009305,49.363660
3,1.050000e+10,3614.424097,0.000025,0.000003,0.000050,50,50.000000,50.009305,49.363660
4,2.500000e+10,1518.058121,0.000025,0.000002,0.000050,50,50.000000,46.674658,46.126636
5,2.500000e+10,1518.058121,0.000005,0.000020,0.000010,50,158.113883,159.638120,146.422610
6,2.500000e+10,1518.058121,0.000010,0.000003,0.000020,50,70.710678,70.714306,69.195615
7,4.500000e+10,843.365623,0.000025,0.000003,0.000050,50,50.000000,50.009305,49.363660
8,1.050000e+10,3614.424097,0.000025,0.000003,0.000050,50,50.000000,50.009305,49.363660
9,2.500000e+10,1518.058121,0.000008,0.000031,0.000016,50,158.113883,158.157206,145.250608


In [49]:
np.array([(0,1), (2,3)]) + np.array([(10, 0)])

array([[10,  1],
       [12,  3]])

In [50]:
def resistance_calculator(heater_length,
                          heater_width,
                          heater_square_resistance = None):
    return heater_square_resistance * heater_length / heater_width
resistance_calculator(93,
                      21,
                      heater_square_resistance = 11.5)

50.92857142857143